In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\MCW\\Documents\\Ajith\\MLOps\\project_02\\Interstate_Traffic_Volume_Project'

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_batch_size: int

In [7]:
from trafficVolumePrediction.constants import *
from trafficVolumePrediction.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/data_trafficVolume.csv",
            all_params=self.params,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
from urllib.parse import urlparse

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        data = pd.read_csv(self.config.training_data)
        x = np.array(data[["holiday","temp","rain_1h","snow_1h","clouds_all","weather_main","weather_description","year","month","day","hour"]])
        y = np.array(data[["traffic_volume"]])
        self.xtrain, self.xval, self.ytrain, self.yval = train_test_split(x, y, 
                                                        test_size=0.20, 
                                                        random_state=33)

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(x=self.xval,
                        y=self.yval,
                        batch_size=self.config.params_batch_size,)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-12-29 22:54:01,438: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-29 22:54:01,444: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-29 22:54:01,445: INFO: common: created directory at: artifacts]

[2023-12-29 22:54:02,673: WARNING: module_wrapper: From c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
20/20 [==============================] - 1s 21ms/step - loss: 5531752.5000 - accuracy: 0.0000e+00
[2023-12-29 22:54:03,426: INFO: common: json file saved at: scores.json]
